In [1]:
import torch
import asyncio
from sentence_transformers import SentenceTransformer
from abc import ABC,abstractclassmethod

class BaseEmbedding(ABC):

    @abstractclassmethod
    def _generate_embeddings(self, text):
        pass
    @abstractclassmethod
    def embed(self, text):
        pass


class SentenceTransformersEmbedding(BaseEmbedding):
    def __init__(self, model_name:str = "sentence-transformers/all-MiniLM-L6-v2", device:str = 'cpu',show_progress:str = False):
        super().__init__()
        self.show_progress = show_progress
        self.model_name = model_name
        self.device = device
        self.model = SentenceTransformer(self.model_name,device=self.device)

    def _generate_embeddings(self, text):
        embeddings = self.model.encode(text,show_progress_bar=self.show_progress).tolist()
        return embeddings
        
    def embed(self, text):
        return self._generate_embeddings(text)

In [14]:
import os
from sentence_transformers import SentenceTransformer
import chromadb
from semantic_text_splitter import HuggingFaceTextSplitter
from tokenizers import Tokenizer
from chromadb import Documents, EmbeddingFunction, Embeddings

class ChromaDB:
    def __init__(self,embed_model = None,show_progress:bool = False):
        if embed_model == None:
            self.embedder = SentenceTransformersEmbedding()
        else:
            self.embedder = embed_model

    def save(self,documents, db_path,collection_name:str = "quickstart"):
        client = chromadb.PersistentClient(path=os.path.abspath(db_path))
        collection = client.create_collection(collection_name,metadata={"hnsw:space": "cosine"})
        embeddings = self._get_document_embeddings(documents)
        collection.add(embeddings=embeddings, documents=documents, ids=[f"id{i}" for i in range(1, len(embeddings) + 1)])
        print(f"ChromaDB saved successfully to {db_path}")
        
    def _get_document_embeddings(self,text):
        return self.embedder.embed(text)
        
    def retrieve(self,user_input,db_path,collection_name = "quickstart",top_k = 1):
        client = chromadb.PersistentClient(path=os.path.abspath(db_path))
        collection = client.get_collection(collection_name)
        # return collection  
        results = collection.query(
        query_embeddings = [self._get_document_embeddings(user_input)],
        n_results=top_k,
        )
        scores = results["distances"][0]
        documents = results["documents"][0]
        ids = results["ids"][0]
        retrieved_data = [[document,score,id] for document,score,id in zip(documents, scores,ids)]
        return retrieved_data


In [19]:
from typing import List
import os
class Ensemble:
    def __init__(self,vector_db,model_names:List[str],embedder = None,device = "cpu"):
        self.vector_db = vector_db
        self.model_names = model_names
        if embedder != None:
            self.embedder = embedder
        else:
            self.embedder = SentenceTransformersEmbedding
        self.device = device
        self.vector_dbs = [self.vector_db(embed_model = self.embedder(model_name=model_name,device = self.device)) for model_name in self.model_names]
        
    def generate_suffix(self,length):
        for i in range(1,length):
            yield i
        
    def save(self,documents,db_path,**kwargs):
        for db,model_name in zip(self.vector_dbs,self.model_names):
            db_path = os.path.join(db_path,f"_{model_name}")
            db.save(documents,db_path,**kwargs)
        print("documents stored successfully")

    def retrieve(self,user_input,db_path,**kwargs):
        result = []
        for db,model_name in zip(self.vector_dbs,self.model_names):
            db_path = os.path.join(db_path,f"_{model_name}")
            context = db.retrieve(user_input,db_path,**kwargs)
            result.append(context)
        return result

In [54]:

ensemble = Ensemble(ChromaDB,["BAAI/bge-large-en-v1.5","llmrails/ember-v1"])

In [55]:
from semantic_text_splitter import HuggingFaceTextSplitter
def segment_text(file_contents, chunk_size):
        tokenizer = Tokenizer.from_pretrained("bert-base-uncased")
        splitter = HuggingFaceTextSplitter(tokenizer, trim_chunks=False)

        sentences = splitter.chunks(file_contents, chunk_size)

        result_chunks = []
        current_chunk = ""
        for i in range(len(sentences)):
            current_chunk += f" {sentences[i]}"
            if current_chunk.count(" ") >= 256:
                result_chunks.append(current_chunk)
                current_chunk = f"{sentences[i-2]} {sentences[i-1]} {sentences[i]}"
        return result_chunks

In [56]:
import requests

def get_response(query,context):
    payload = {
      "message_content": query,
      "content": context,
      "company_name": "grandcholan",
      "domain": "foods and services"
    }
    
    response = requests.post("http://46.43.144.145:8078/rulebased_bot",json=payload)
    
    return response.json()
     

In [57]:
with open("grand_cholan.txt") as f:
    text = f.read()

text = segment_text(text,chunk_size=50)

In [58]:
ensemble.save(text,"./grandcholan1")

ChromaDB saved successfully to ./grandcholan1/_BAAI/bge-large-en-v1.5
ChromaDB saved successfully to ./grandcholan1/_BAAI/bge-large-en-v1.5/_llmrails/ember-v1
documents stored successfully


In [64]:
ensemble.retrieve("hi","./grandcholan1")


[[['நாடு தனித்து இருந்தது; \xa0 மேற்கே சேரநாடு, கங்க நாடு, \xa0குடகு, நுளம்பாடி, \xa0தடிகை  பாடி, \xa0மேல் கடற்கரை நாடு முதலியன தனித்தனி சிற்றரசர்களாக இருந்தனர். \xa0  வடமேற்கே இராட்டிரகூடரை அழித்துப் புதிய பேரரசை \xa0 இரட்டபாடியில் \xa0அமைத்த மேலைச் சாளுக்கியர் ஆண்டுவந்தனர். \xa0இலங்கையில் ஐந்தாம் மகிந்தன் ஆண்டுவந்தான்.\nராஜராஜ சோழன் போர்கள்\n காந்தளூர் சாலைப் போர்\n இராசராச சோழன் \xa0பதவியேற்ற பின்பு \xa0இவர்களது பகைவர்களான \xa0 சேரர் \xa0பாண்டியர், \xa0சிங்களர்களை வெற்றி பெறும் நோக்குடன் நிலையான படையை அமைக்கும் முயற்சியில் ஈடுபட்டார் . \xa0எட்டாண்டு கால ஏற்பாட்டுக்கு பின்னர் படையெடுப்புகளுக்கு தயாரானார்.\n இராசராச சோழன் முதற்கட்டமாக சேர நாட்டின் மீது படையெடுக்கும் நோக்குடன் \xa0சேர நாட்டின் \xa0கடற்படை என்பதால் \xa0மிக பலமான கடற்படையை \xa0உருவாக்கினார். \xa0  தனது திட்டத்தின்படி, \xa0 தான் அனுப்பிய \xa0தூதுவரை சேரமன்னன் பாஸ்கர \xa0ரவிவர்மன் \xa0கைது செய்து விட்டமையால்  சேர நாட்டின் மீது படையெடுத்தார். \xa0திருவனந்தபுரத்தில் உள்ள காந்தளூர் சாலை என்னுமிடத்தில் போர் மூண்டது. \xa0 இப்போ

In [68]:
query = "price of idli"

top_two_model_data = ensemble.retrieve(query,"./grandcholan1",top_k = 2)

chunks  = set()
for data in top_two_model_data:
    for chunk in data:
        chunks.add(chunk[0])

temp_chunk = "\n".join(chunks)


In [69]:
temp_chunk

"(Aromatic Kadai spices cooked with bell peppers, onions and tomato)\nBhuna\n(Rich thick sauce cooked with garlic, onions and tomato)\nSaag\n(Chopped spinach based curry with ginger, garlic, onions and tomato)\n Indian Meat & Fish Curry\nVillage Lamb Shank Curry\n£17.50\n(Lamb shank cooked with chef’s special South Indian spices)\nMango King prawn Curry\n£14.95\n (King prawns cooked with raw mango pieces, coconut and South Indian spices)\nKerala Fish Curry (King Fish / Tilapia)\n£14.95\n(Fish curry cooked with onion, tomato, tamarind and coconut milk)\n Cholan Chettinad Chicken Curry\n£13.95\n(Chicken thigh curry cooked with South Indian spices)\nKashmiri Lamb Rogan Josh\n£14.95\n(Diced lamb slow cooked with special Kashmiri chilli & spices)\nPunjabi Butter Chicken\n £13.95\n(Tandoori chicken cooked with tomato-based mild and creamy curry)\nIdli, Vada & Dosa\nIdli\n£7.95\n (Fermented rice and dal batter, steamed in a unique idli pot.)\nMedhu Vada\n£5.95\n(Battered Urad dal mixed with c

In [70]:
get_response(query,temp_chunk)

{'intent': '', 'response': 'Idli costs £7.95.'}